In [ ]:
#setup 
import os
import datetime
import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
!pip install scikit-learn
import sklearn
import pandas as pd
!pip install geopandas
!pip install geojson
!pip install shapely
import geopandas as gpd
import geojson
import geopandas as gpd
from fiona.crs import from_epsg
import os, json
from shapely.geometry import shape, Point, Polygon, MultiPoint
from geopandas.tools import sjoin
import matplotlib.cm as cm
import matplotlib.pyplot as plt # plotting
import seaborn as sns; sns.set()
from IPython.display import Image
#from branca.colormap import  linear
import json
#import branca.colormap as cm
!pip install folium
import folium

mpl.rcParams['figure.figsize'] = (4, 3)
mpl.rcParams['axes.grid'] = False


In [ ]:
new_table = pd.read_csv('https://raw.githubusercontent.com/giobbu/App_Traff_Forecast_DeapLearn/master/data/Flow_BEL_street_30min.csv')

In [ ]:
df_belgium = gpd.read_file('https://raw.githubusercontent.com/giobbu/App_Traff_Forecast_DeapLearn/master/data/Belgium_streets.json')

In [ ]:
# this source: 
# the https://www.kaggle.com/code/giobbu/seasonal-persistence-model/notebook#Results-Comparison-between-Seasonal-model-(baseline)-and-LSTM-encoder-decoder-model
# selects the roads that have an average traffic flow of 10 or larger, we also do this, this code is from the notebook listed above 

table_index = new_table.iloc[:,1:]
ALL_STREETS = list(table_index.columns.values)

mean_flow =[]
new_street=[]

mean_value = 20

for street in ALL_STREETS:
    
    single_street=table_index[street]
    mean = np.mean(single_street)
    mean_flow.append(mean)
    new_street.append(street)
    
    
df_mean_flow = pd.DataFrame({'street_index':new_street, 'mean_flow': mean_flow})
print('')
print(df_mean_flow.head())
print('')

STREETS = df_mean_flow[(df_mean_flow['mean_flow'] >= mean_value)] 
STREETS = STREETS.sort_values(by=['street_index'])
STREETS = list(STREETS.street_index)

keys_dim = 1683
keys_dim_time = 1686

print('considering a average traffic flow of ' + str(mean_value)+' per street')
print('')
print('mean traffic flow '+str(mean_value)+ ' ---> number of street segments: ' + str(len(STREETS)))
print(len(STREETS))

In [ ]:
#this is for selecting street indecies for extension 
def in_area(poly):
  bounds = poly.bounds
  lon_lat_list = [[ 4.14878,50.35045], [4.14878,50.35045], [4.54878,50.35045 ], [4.54878,51.05045], [4.14878,51.05045]]
  min_x = lon_lat_list[0][0]
  max_x = lon_lat_list[2][0]
  min_y = lon_lat_list[0][1]
  max_y = lon_lat_list[3][1]

  if (((bounds[0]<= max_x and bounds[0] >=min_x) or (bounds[2]<= max_x and bounds[2] >=min_x))and
      ((bounds[1]<= max_y and bounds[1] >=min_y) or (bounds[3]<= max_y and bounds[3] >=min_y))):  
      return True 
  return False 

streets_in_area = []
df_belgium['is_in_area']  = df_belgium.apply(lambda x: in_area(x['geometry']),axis = 1)
the_streets = df_belgium[df_belgium['is_in_area'] ==True] 

STREETS2 = the_streets.index.tolist()
len(STREETS2)
STREETS2 = list(map(str, STREETS2))
STREETS2 = [x + '.0' for x in STREETS2]
#STREETS = list(set(STREETS) & set(STREETS2))
print(len(STREETS))
#keys_dim = 1042


In [ ]:
new_table['datetime'] = pd.to_datetime(new_table['datetime'])

In [ ]:
plt.plot(new_table['datetime'],new_table.mean(axis = 1))

In [ ]:
from tensorflow.python.ops.gen_array_ops import fake_quant_with_min_max_vars_per_channel
#we need to create "windows" in the data --> 
#X: input tensor : use a window of size s, lets say 5, as X
#Y: output vector : traffic for hour six predicted based on the previous window 

#this is the window function for multiple input features and multiple output pairs
#what's the difference between df_to_x_y2 and df_to_x_y --> the first one is just for one input vector (ie the data from one street) this 
#version accepts many columns of data (all the street values and the transformed time data)
def df_to_x_y2(positional_data,non_positional, window_size): 
  #df_as_np = df.to_numpy()
  print(positional_data.shape)
  print(non_positional.shape)
  X = []
  Y = []
  for i in range((len(positional_data))-window_size-1):
    row = [r for r in positional_data[i:i+window_size]]
    
    #The line below makes the last value in the batch 0 (relative to what we are predicting, 
    #traffic flow rate, the positional information is still encoded )
    row.append(positional_data[i+window_size+1] - non_positional[i+window_size+1]) 
    X.append(row)
    
    label = [non_positional[i+window_size][0:]] 
    Y.append(label)
  return np.array(X), np.array(Y)

In [ ]:
#this is for transforming datetime to recognizable inputs 
#sin and cos transformation for hour of the day 
df = new_table
df = df[STREETS + ['datetime']] 
 #this removes all of the streets with aveage flow <10 

df.head()

time = df['datetime'].to_numpy()

In [ ]:
def positional_encoding(df):
  import datetime as dt
  from sklearn.preprocessing import MinMaxScaler
  df = pd.Series(df)
  hour = df.dt.hour.to_numpy().astype('float32')
  hour_sin = np.sin(hour*2* np.pi/23)
  hour_cos = np.cos(hour*2* np.pi/23)
  
  day_of_week = df.dt.dayofweek.to_numpy()
  day_of_week_scaler = MinMaxScaler(feature_range=(-1, 1))
  day_of_week = day_of_week.reshape(-1, 1)
  day_of_week = day_of_week_scaler.fit_transform(day_of_week)
  day_of_week = day_of_week[:,0]
  
  pos_encoding_val = hour_sin + hour_cos + day_of_week
  pos_encoding_val = pos_encoding_val.reshape(-1,1)
  final_scaler = MinMaxScaler(feature_range=(0,1))
  pos_encoding_val = final_scaler.fit_transform(pos_encoding_val)
  return pos_encoding_val
  

In [ ]:
#test positoinal_encoding function 
positional_encoding(time)

In [ ]:
class PositionalEmbedding(tf.keras.layers.Layer):
  def __init__(self, date_time_info,time_weight):
    super().__init__()
    from sklearn.preprocessing import MinMaxScaler
    self.time_weight = time_weight
    self.scaler = MinMaxScaler(feature_range=(0,1))
    self.pos_encoding = positional_encoding(date_time_info)

  def call(self, x):
    print(x.shape)
    print(type(x))
    print(x)
    length = tf.shape(x)[1]
    x = self.scaler.fit_transform(x)
    # This factor sets the relative scale of the embedding and positonal_encoding.
    print(self.pos_encoding.shape)
    x_positioned = np.add(x*(1-self.time_weight), (self.time_weight * self.pos_encoding))
    print(x.shape)
    x_final,y_final= df_to_x_y2(x_positioned,x,window_size=12) 
    x_final = tf.constant(x_final)
    y_final = tf.constant(y_final)
    return x_final,y_final

In [ ]:
#testing the positional embedding class 
positional_em = PositionalEmbedding(time,0.4)
positional_em2 = PositionalEmbedding(time,0)

print('shape1',positional_em(df[STREETS].to_numpy())[0][0])
print('shape2',positional_em2(df[STREETS].to_numpy())[0][0])

x1_df = df[STREETS + ['datetime']].to_numpy()

In [ ]:
class BaseAttention(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
    super().__init__()
    self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
    self.layernorm = tf.keras.layers.LayerNormalization()
    self.add = tf.keras.layers.Add()

In [ ]:
class CrossAttention(BaseAttention):
  def call(self, x, context):
    attn_output, attn_scores = self.mha(
        query=x,
        key=context,
        value=context,
        return_attention_scores=True)
   
    # Cache the attention scores for plotting later.
    self.last_attn_scores = attn_scores

    x = self.add([x, attn_output])
    x = self.layernorm(x)

    return x

In [ ]:
#in globalself attention, we are comparing every token to iteself within the same sequence of tokens, here the query and value are both = x
class GlobalSelfAttention(BaseAttention):
  def call(self, x):
    attn_output = self.mha(
        query=x,
        value=x,
        key=x)
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x

In [ ]:
class CausalSelfAttention(BaseAttention):
  def call(self, x):
    attn_output = self.mha(
        query=x,
        value=x,
        key=x,
        use_causal_mask = True)
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x

In [ ]:
class FeedForward(tf.keras.layers.Layer):
  def __init__(self, d_model, dff, dropout_rate=0.1):
    super().__init__()
    self.seq = tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),
      tf.keras.layers.Dense(d_model),
      tf.keras.layers.Dropout(dropout_rate)
    ])
    self.add = tf.keras.layers.Add()
    self.layer_norm = tf.keras.layers.LayerNormalization()

  def call(self, x):
    x = self.add([x, self.seq(x)])
    x = self.layer_norm(x) 
    return x

In [ ]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self,*, d_model, num_heads, dff, dropout_rate=0.1):
    super().__init__()

    self.self_attention = GlobalSelfAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)
    
    self.causal_self_attention = CausalSelfAttention(
      num_heads=num_heads,
      key_dim=d_model,
      dropout=dropout_rate
    )

    self.ffn = FeedForward(d_model, dff)

  def call(self, x):
    print('eh',x.shape)
    x = self.self_attention(x)
    print(x.shape)
    x = self.causal_self_attention(x)
    x = self.ffn(x)
    return x

In [ ]:
x1_df_test = x1_df[0:336]
x1_df_test_time = x1_df_test[:,-1]
x1_test_data = df[STREETS][0:336].to_numpy()
print()
positional_em = PositionalEmbedding(x1_df_test_time,0.4)
x1_test, y1_test = positional_em(x1_test_data)
print(x1_test.shape)

In [ ]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, *, num_layers, d_model, num_heads,
               dff, dropout_rate=0.1):
    super().__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.enc_layers = [
        EncoderLayer(
                    d_model=d_model,
                     num_heads=num_heads,
                     dff=dff
                     )
        for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(dropout_rate)

  def call(self, x):
    # `x` is token-IDs shape: (batch, seq_len)
    print('try embedd',x.shape)

    # Add dropout.
    x = self.dropout(x)

    for i in range(self.num_layers):
      print()
      x = self.enc_layers[i](x)

    return x  # Shape `(batch_size, seq_len, d_model)`.

In [ ]:
sample_encoder = Encoder(num_layers=2,
                         d_model=keys_dim,
                         num_heads=5,
                         dff=keys_dim
                         )

sample_encoder_output = sample_encoder(encoder_test, training=False)

# Print the shape.
print(encoder_test.shape)
print(sample_encoder_output.shape)  # Shape `(batch_size, input_seq_len, d_model)`.

In [ ]:
#encoder only transformer 
class Transformer(tf.keras.Model):
  def __init__(self, *, num_layers, d_model, num_heads, dff, dropout_rate=0.1):
    super().__init__()
    self.encoder = Encoder(num_layers=num_layers, d_model=d_model,
                           num_heads=num_heads, dff=dff,
                           dropout_rate=dropout_rate)
    #flattening before the final layer 
    self.flatten = tf.keras.layers.Flatten()
    self.final_layer = tf.keras.layers.Dense(d_model)

  def call(self, inputs):
    # To use a Keras model with `.fit` you must pass all your inputs in the
    # first argument.
    print('input_shape',inputs.shape)
    x = inputs
    x = self.encoder(x)  # (batch_size, target_len, d_model)
    

    # Final linear layer output.
    #x = self.flatten(x)
    x = x[:,-1,:]
    logits = self.final_layer(x)  # (batch_size, target_len, target_vocab_size)

    

    # Return the final output and the attention weights.
    return logits

In [ ]:
num_layers = 2
d_model = keys_dim
dff = keys_dim
num_heads = 5
dropout_rate = 0.1



In [ ]:
transformer = Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
 
    dropout_rate=dropout_rate)




In [ ]:
print(x1_df.shape)
length = 2832
#80, 10, 10 split from train val test 
val_step = int(0.1 * length) * 2
test_step = int(0.1 * length)

val_s = length - val_step
test_s = length - test_step

x_train = x1_df[:val_s]
x_val = x1_df[val_s:test_s]
x_test= x1_df[test_s:]



print(x_test.shape)

In [ ]:
#positionally encode and divide all data sets into batches 
train_time = x_train[:,-1]
train_feat = x_train[:,:-1]
val_time = x_val[:,-1]
val_feat = x_val[:,:-1]
test_time = x_test[:,-1]
test_feat = x_test[:,:-1]
print(x_test.shape)
print(x_val.shape)
print(test_time.shape)

train_pos_enc = PositionalEmbedding(train_time,0.4)
val_pos_enc = PositionalEmbedding(val_time,0.4)
test_pos_enc = PositionalEmbedding(test_time,0.4)


x_train,y_train = train_pos_enc(np.asarray(train_feat).astype('float32'))
x_val,y_val = val_pos_enc(np.asarray(val_feat).astype('float32'))
x_test,y_test = test_pos_enc(np.asarray(test_feat).astype('float32'))


In [ ]:
plt.plot(x_train[0,:,2])

plt.scatter(12,y_train[0,:,2])
print(y_train[0,:,2])
print(x_train[0,:,2])

In [ ]:
train_dataloader = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(64)
val_dataloader = tf.data.Dataset.from_tensor_slices((x_val, y_val)).shuffle(10000).batch(64)
test_dataloader = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(64)

train_batch_size = 2253
val_batch_size = 270
test_batch_size = 270

print(train_dataloader)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [ ]:
def evaluate(dataloader, model, log=False):
    
    for traffic_batch, labels_batch in dataloader:
        traffic = traffic_batch
        print(traffic.shape)
        predict_flow = model(traffic)
        mse = tf.keras.losses.MeanSquaredError()
        mse = mse(labels_batch,predict_flow ).numpy()
    
    return predict_flow

In [ ]:

evaluate(test_dataloader,transformer)

In [ ]:
def loss(y_true, y_pred):
    mse = tf.keras.losses.MeanSquaredError()
    y = mse(y_true, y_pred)
    return y_train

In [ ]:
transformer.compile(optimizer=optimizer, 
                  loss = tf.keras.losses.MeanAbsoluteError(), 
                  metrics= [tf.keras.losses.MeanAbsoluteError()]) 

In [ ]:
transformer.fit(train_dataloader,epochs = 5,validation_data = val_dataloader)

In [ ]:
response = evaluate(test_dataloader,transformer)

In [ ]:
res = transformer.predict(x_test)
print(y_test.shape)
res[0]

In [ ]:
#plotting specific streets 
plt.rcParams["figure.figsize"] = (20,4)
plt.plot(np.arange(300,400),((y_test[80,:,:][0][300:400])),label = 'Actual Values')
plt.xlabel('Street Identifier')
plt.ylabel('Traffic Conjestion Rate, Scaled Between (0-1)')
plt.title('Predicted vs Actual Conjestion Rates for input batch #80')

plt.plot(np.arange(300,400),(res[80][300:400]), label = 'Prediction')
plt.legend()

In [ ]:
#checking performance for low actal rates and high actual rates 
plt.rcParams["figure.figsize"] = (20,4)
high_volume = tf.reduce_mean(y_test,2)
high_volume = pd.DataFrame(high_volume.numpy(),columns = ['flow_rate'])
def is_high_vol(row,vol):
    if row['flow_rate'] <= vol:
        return 'low'
    return 'high'
high_volume['h/l'] = high_volume.apply(lambda x: is_high_vol(x,average_flow),axis = 1)
high_volume_batches = (high_volume.index[high_volume['h/l']=='high'] ).tolist()
high_volume_batches
low_volume_batches = (high_volume.index[high_volume['h/l']=='low'] ).tolist()

#actual low and high averages 
high_vol_y_test = tf.gather(y_test,high_volume_batches)
low_vol_y_test = tf.gather(y_test,low_volume_batches)
low_ave = tf.reduce_mean(low_vol_y_test,0)
low_ave_select = low_ave[0][0:150]
high_ave = tf.reduce_mean(high_vol_y_test,0)
high_ave_select = high_ave[0][0:150]

#prediction high and low averages
high_vol_pred = tf.gather(res,high_volume_batches)
low_vol_pred = tf.gather(res,low_volume_batches)
low_ave_pred = tf.reduce_mean(low_vol_pred,0)
print(low_ave_pred.shape)
low_ave_pred_select = low_ave_pred[0:150]
high_ave_pred = tf.reduce_mean(high_vol_pred,0)
high_ave_pred_select = high_ave_pred[0:150]

plt.plot(scale(high_ave_select), label = 'Actual')
plt.plot(scale(high_ave_pred_select),label = 'Prediction')
plt.xlabel('Street Identifier')
plt.ylabel('Traffic Conjestion Rate, Scaled Between (0-1)')
plt.title('Batches with normalized average flow >= average_flow (Prediction and Actual values normalized between 0 and 1)')


plt.legend(loc = 'upper right')
print(high_volume_batches)

In [ ]:
#calculating mean_abs_error for high vs low samples 
#high 
high_ave_select2 = high_ave[0][0:]
high_ave_pred_select2 = high_ave_pred
high_mae = tf.keras.losses.MeanAbsoluteError()
h_mae = high_mae(high_ave_select2, high_ave_pred_select2).numpy()

#low
low_ave_select2 = low_ave[0][0:]
low_ave_pred_select2 = low_ave_pred
low_mae = tf.keras.losses.MeanAbsoluteError()
l_mae = low_mae(low_ave_select2, low_ave_pred_select2).numpy()

print(h_mae,l_mae)

In [ ]:
def scale(input):
    return tf.divide(tf.subtract(input,tf.reduce_min(input)),tf.subtract(tf.reduce_max(input),tf.reduce_min(input)))
    

In [ ]:
res2 = tf.reduce_mean(res,1)
actual2 = tf.reduce_mean(y_test,2)
res2_scaled = tf.divide(tf.subtract(res2,tf.reduce_min(res2)),tf.subtract(tf.reduce_max(res2),tf.reduce_min(res2)))
actual2_scaled = tf.divide(tf.subtract(actual2,tf.reduce_min(actual2)),tf.subtract(tf.reduce_max(actual2),tf.reduce_min(actual2)))

In [ ]:
plt.plot(res2_scaled)
#plt.plot(actual)
plt.plot(actual2_scaled)


In [ ]:
plt.plot((res[:,10]))


In [ ]:
plt.plot(scale(res[:,1000]))
plt.plot(scale(y_test[:,0,1000]))
print((y_test.shape))

In [ ]:
#MSE low vs high flow rates 
average_flow = tf.reduce_mean(tf.reduce_mean(y_test,2),0)

In [ ]:
average_flow